In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os,sys
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from deeplabv3 import DeepLabV3

In [3]:
model = DeepLabV3(
        n_classes=21,
        n_blocks=[3, 4, 23, 3],
        atrous_rates=[6, 12, 18],
        multi_grids=[1, 2, 4],
        output_stride=8,
    )
print(model)


DeepLabV3(
  (layer1): _Stem(
    (conv1): _ConvBnReLU(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
  )
  (layer2): _ResLayer(
    (block1): _Bottleneck(
      (reduce): _ConvBnReLU(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (conv3x3): _ConvBnReLU(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (increase): _ConvBnReLU(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fals

In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

In [5]:
import torchvision.utils

In [6]:
from collections import defaultdict
import torch.nn.functional as F
import torch
from loss import dice_loss

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)
        
    pred = torch.sigmoid(pred)
    dice = dice_loss(pred, target)
    
    loss = bce * bce_weight + dice * (1 - bce_weight)
    
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    
    return loss

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

In [8]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import confusion_matrix

plt.ion()   # interactive mode

In [12]:
from PIL import Image
class Dataset(object):
    def __init__(self, datapath):
        # load paths for your dataset and put in self.data
        self.datapath = datapath
        self.maskpath = '../masks'
        self.data = []
        self.masks = []
        for folder in os.listdir(datapath):
            for filename in os.listdir(datapath+'/'+folder):
                self.data.append(folder+'/'+filename)
                self.masks.append(filename.split('.')[0])
            
            
    def __getitem__(self, idx):
        # load images
        image = Image.open(self.datapath + '/' +  self.data[idx]).convert("RGB").resize((672,672))
        image_size = (image.size[0], image.size[1], 3)
        image = np.array(image.getdata()).reshape(image_size)
        maskname = self.maskpath + '/' +  self.masks[idx] + '_mask.jpg'
        
        mask_size = (85,85)

        if self.masks[idx] + '_mask.jpg' in os.listdir(self.maskpath):
            mask = np.array(Image.open(maskname).convert("RGB").resize(mask_size).getdata())
        elif self.masks[idx] + '_mask0.jpg' in os.listdir(self.maskpath):
            i = 0
            mask = np.zeros((mask_size[0],mask_size[1],3)).reshape([-1,3])
            while self.masks[idx] + '_mask' + str(i) + '.jpg' in os.listdir(self.maskpath):
                maskname = self.maskpath + '/' +  self.masks[idx] + '_mask' + str(i) + '.jpg'
                mask_ = np.array(Image.open(maskname).convert("RGB").resize(mask_size).getdata())
                mask = mask + mask_ # can add since masks should not overlap
                i += 1
        else:
            mask = np.zeros((mask_size[0],mask_size[1],3))

        mask = mask.reshape((mask_size[0],mask_size[1],3))
        mask = mask.mean(axis=2)
        mask = mask.reshape([1, mask_size[0], mask_size[1]])
        
        image = image.transpose([2,0,1])
        
        #two size options were reshaped to (672,672,3)
        #(1680, 1050, 3)
        #(640, 480, 3)
        
        return torch.Tensor(image), torch.Tensor(mask)

    def __len__(self):
        return len(self.data)

In [13]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation((-45, 45)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(), 
        transforms.ToTensor(),
        transforms.ColorJitter(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../dataset'
image_datasets = {x: Dataset(os.path.join(data_dir, x))
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, 
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device('cpu')#"cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device('cpu')#"cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = DeepLabV3(
        n_classes=num_class,
        n_blocks=[3, 4, 23, 3],
        atrous_rates=[6, 12, 18],
        multi_grids=[1, 2, 4],
        output_stride=8,
    ).to(device)

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)        
        
model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=1)

cpu
Epoch 0/0
----------
LR 0.0001


In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp /= 255
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def imshowimagemasked(img, mask, title=None):
    """Imshow for Tensor."""
    inp = img
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.imshow(mask.numpy()/255,cmap='jet',alpha=0.2)
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def visualize_segmenter(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                imshow(inputs.cpu().data[j])
                imshowimagemasked(inputs.cpu().data[j], preds.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_segmenter(model)